# Number of causal sites

This notebook uses the French Canadian dataset and the inferred tree sequence from the 1000 Genomes Project to examine how the number of causal sites is influencing the simulation speed.

### Dataset:

The datasets are obtained from the following links:

- French Canadian dataset (`simulated_genomes_chr9.tsz`) is installed from https://zenodo.org/record/6839683
- 1000 Genomes Project dataset (`1kg_chr9.trees.tsz`) is installed from https://zenodo.org/record/3051855

Please put the datasets in `data` folder before running the codes.

In [1]:
import tstrait
import time
import numpy as np
import pandas as pd
import tszip
import msprime

In [2]:
# Run this code before measuring the computational time, as tstrait uses numba
ts = msprime.sim_ancestry(samples=1000, recombination_rate=1e-8, sequence_length=10_000, population_size=10_000,)
ts = msprime.sim_mutations(ts, rate=1e-8)
trait_model = tstrait.trait_model(distribution="normal", mean=0, var=1)
sim_result = tstrait.sim_phenotype(ts, num_causal=1, model=trait_model, h2=0.3)

In [3]:
# This code simulates traits for num_rep times by using tstrait, and returns the array of simulation time
def compute_time_tstrait(ts, num_causal, num_rep):
    times = []
    trait_model = tstrait.trait_model(distribution="normal", mean=0, var=1)
    for _ in range(num_rep):
        before = time.perf_counter()
        sim_result = tstrait.sim_phenotype(ts, num_causal=num_causal, model=trait_model, h2=0.3)
        duration = time.perf_counter() - before
        times.append(duration)
    return times

## French Canadian dataset

We will be using chromosome 9 data in the French Canadian dataset. We will be measuing the computational time for 100 causal sites.

In [4]:
# French Canadian dataset
ts = tszip.decompress("data/simulated_genomes_chr9.tsz")

In [5]:
# Number of individuals
ts.num_individuals

2723339

In [6]:
# Number of sites
ts.num_sites

48896598

In [7]:
french_canadian_time = compute_time_tstrait(ts=ts, num_causal=100, num_rep=5)

In [8]:
french_canadian_time

[90.01081990008242,
 80.33812760002911,
 74.56985790003091,
 77.97003110009246,
 80.5568057000637]

In [9]:
np.mean(french_canadian_time)

80.68912844005972

# 1000 Genomes Dataset

We will be using chromosome 9 data in the 1000 Genomes dataset. We will be measuing the computational time for 1000 causal sites.

In [10]:
# 1000 Genomes project
ts = tszip.decompress("data/1kg_chr9.trees.tsz")

In [11]:
# Number of individuals
ts.num_individuals

2504

In [12]:
# Number of sites
ts.num_sites

1685401

In [13]:
thousand_genome_time = compute_time_tstrait(ts=ts, num_causal=100, num_rep=5)

In [14]:
thousand_genome_time

[5.869191699894145,
 5.408523800084367,
 5.1522061000578105,
 5.103017200017348,
 5.487022200017236]

In [15]:
np.mean(thousand_genome_time)

5.403992200014182